In [3]:
import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew, t, ttest_1samp, norm

Problem 1
Remember from last week we discussed that skewness and kurtosis functions in statistical packages are often biased. Is your function biased? Prove or disprove your hypothesis.

Steps
1. Sample 10, 100, 1,000, 10,000 standardized random normal values.
2. Calculate the skewness and kurtosis
3. Sample the skewness and kurtosis by repeating steps 1 and 2 100 times.
4. Calculate the T statistic and the p-value (μ0 = 0).
5. If the value is lower than your threshold (typically 5%), then you reject the hypothesis that the kurtosis function is unbiased.

In [12]:
def tTest(sampleNum, target): 
    times = 100

    samples = np.empty(times)
    for i in range(times):
        if target == 'skewness':
            samples[i] = skew(np.random.normal(0, 1,sampleNum))
        elif target == 'kurtosis':
            samples[i] = kurtosis(np.random.normal(0, 1,sampleNum))

    t_stat, p_val = ttest_1samp(samples, popmean=0)

    if p_val < 0.05:
        print("{} p-value is {}.\nReject the null hypothesis.\n" .format(target, p_val))
    else:
        print("{} p-value is {}.\nFail to reject the null hypothesis.\n" .format(target, p_val))
    return

In [13]:
sample = [10, 100, 1000, 10000]

for num in sample:
    print("{} samples: " .format(num))
    tTest(num, 'skewness')
    tTest(num, 'kurtosis')

10 samples: 
skewness p-value is 0.767314189748661.
Fail to reject the null hypothesis.

kurtosis p-value is 2.555351592378225e-15.
Reject the null hypothesis.

100 samples: 
skewness p-value is 0.9567289152766983.
Fail to reject the null hypothesis.

kurtosis p-value is 0.916747622125282.
Fail to reject the null hypothesis.

1000 samples: 
skewness p-value is 0.247015894960639.
Fail to reject the null hypothesis.

kurtosis p-value is 0.6069190122107347.
Fail to reject the null hypothesis.

10000 samples: 
skewness p-value is 0.8445143483980342.
Fail to reject the null hypothesis.

kurtosis p-value is 0.742155836791117.
Fail to reject the null hypothesis.

